# Study colors in ALMA

In [10]:
import sys
sys.path.append("../../")

import pandas as pd
from src.colors import color_alma as color

path="../../data/interim/ALMA/"

## Collect colors by instance

In [13]:
FILES=!ls $path | grep -v count

In [14]:
acc=[]

for f in FILES:
    temp_dict={}
    instance=pd.read_csv(path+f)
    instance['color'] = instance['logtext'].apply(color)
    for i, r in instance.iterrows():
        if r['color'] not in temp_dict.keys():
            temp_dict[r['color']] = 1
        else:
            temp_dict[r['color']] += 1
    for k in temp_dict.keys():
        acc.append( {'file': f, 'color': k, 'count': temp_dict[k]} )

colors_by_instance = pd.DataFrame(acc)

## Colors statistics

In [15]:
# of instances:
len(FILES)

630

In [16]:
# of colors
len(colors_by_instance["color"].unique())

171

In [17]:
# Normalize by # of instaces
colors_by_instance['count-norm'] = colors_by_instance['count'] / len(FILES)

In [18]:
# Normalized totals by color (AVERAGE!)
norm_total = colors_by_instance[["color", "count-norm"]].groupby(["color"]).sum().sort_values('count-norm', ascending=False)
norm_total

,count-norm
color,
Receiver band was already locked {} {}e {},43.012698
Parking Calibration Device,41.933333
Set coarse tune {} High IFTP {} {},30.473016
Modulation voltage {} {} IFTotal Power {} {} LPR power {} {} Photomixer Current {} {},29.682540
Loop ongoing j {},28.814286
...,...
Field alma CONTROL ANTxx LORR TE_LENGTH archive_delta_percent does not exist,0.001587
Field alma CONTROL ANTxx LORR CAN_ERROR_COUNT archive_delta_percent does not exist,0.001587
Field alma CONTROL ANTxx LORR CAN_ERROR archive_delta_percent does not exist,0.001587


In [24]:
# Filtering total above THRESHOLD_MIN_AVG_COUNT
THRESHOLD_MIN_AVG_COUNT = 0.1
norm_total[ norm_total['count-norm'] >= THRESHOLD_MIN_AVG_COUNT ]

,count-norm
color,
Receiver band was already locked {} {}e {},43.012698
Parking Calibration Device,41.933333
Set coarse tune {} High IFTP {} {},30.473016
Modulation voltage {} {} IFTotal Power {} {} LPR power {} {} Photomixer Current {} {},29.682540
Loop ongoing j {},28.814286
...,...
Limiting subreflector position {} {} {} {} {} mm ensure it does not exceed limits {} {} {} mm,0.233333
Aborting data collection that started {} {} {} {} and was due stop {} {} {} {} Time now {} {} {} {},0.160317
Data acquisition aborted,0.155556


In [ ]:
#colors_by_instance[["color", "count"]].groupby(["color"]).median().sort_values('count', ascending=False)[:50]